# Processamento do PIB Municipal
Este notebook processa imagens de PIB municipal utilizando OCR e gera um CSV consolidado.

### Dados manuais extraídos dos documentos (valores em R$)
Fonte: https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html


In [14]:
# Importação de bibliotecas necessárias
import os
import re
import unicodedata
import pandas as pd
import easyocr

## Criação da pasta de saída
Certifique-se de que a pasta de saída para os dados gerados existe.

In [15]:
# Criação da pasta de saída
os.makedirs("data/generated", exist_ok=True)

imagens = [
        ("data/raw/", "pib_alto_parnaiba.png"),
        ("data/raw/", "pib_balsas.png"),
        ("data/raw/", "pib_tasso_fragoso.png"),
    ]

## Função para normalizar strings
Remove acentos, pontuações extras e converte para letras minúsculas.

In [16]:
def strip_accents_and_punct(s: str) -> str:
    s = ''.join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    s = re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
    return s.lower().strip()

## Função para processar texto OCR
Recebe o texto OCR e retorna os dados estruturados.

In [17]:
def parse_municipio(text: str) -> dict:
    """
    Recebe o texto OCR e retorna:
      - municipio
      - codigo_ibge
      - pib_precos_correntes
      - impostos_liquidos
      - pib_per_capita
      - vab_total
      - vab_agro
      - vab_industria
      - vab_servicos
      - vab_adm_publica
    """
    linhas = [l.strip() for l in text.splitlines() if l.strip()]
    dados = {}

    # —————————————
    # 1) município e código IBGE na 1ª linha
    m = re.match(r"(.+?)\s+c[oó]digo[: ]+(\d+)", linhas[0], re.IGNORECASE)
    if m:
        dados["municipio"] = m.group(1).strip()
        dados["codigo_ibge"] = m.group(2).strip()

    # —————————————
    # 2) mapeia substrings para as colunas
    mapa = {
        "pib a precos correntes":  "pib_precos_correntes",
        "impostos":                "impostos_liquidos",
        "pib per capita":          "pib_per_capita",
        "valor adicionado bruto":  "vab_total",
        "agropecuaria":            "vab_agro",
        "industria":               "vab_industria",
        "servicos":                "vab_servicos",
        "administracao":           "vab_adm_publica",
    }

    # percorre cada linha procurando a chave
    for i, linha in enumerate(linhas[1:], start=1):
        chave_raw = linha
        chave = strip_accents_and_punct(chave_raw)

        for key, col in mapa.items():
            if key in chave:
                # tenta pegar número na mesma linha
                txt = linha.replace(" ", "")
                mnum = re.search(r"([\d\.,]+)", txt)
                # ou na próxima
                if not mnum and i+1 < len(linhas):
                    mnum = re.search(
                        r"([\d\.,]+)", linhas[i+1].replace(" ", ""))
                if mnum:
                    num_txt = mnum.group(1).replace(".", "").replace(",", ".")
                    try:
                        valor = float(num_txt)
                    except ValueError:
                        valor = num_txt
                else:
                    valor = None

                dados[col] = valor
                print(f"  • Encontrou {col}: {valor}")
                break

    return dados

## Iniciar processo da criação dos Dados de Pib Municipal
Utiliza OCR para extrair texto das imagens e processa os dados.

In [ ]:
def gerar_dados_pib():
    reader = easyocr.Reader(["pt"], gpu=False)
    registros = []

    for folder, filename in imagens:
        path = os.path.join(folder, filename)
        print(f"\n→ Processando {filename}…")
        texto = "\n".join(reader.readtext(path, detail=0, paragraph=True))
        print("→ Texto bruto OCR:\n", texto, "\n")
        rec = parse_municipio(texto)
        print("→ Registro extraído:", rec)
        registros.append(rec)

    # monta DataFrame e salva
    df = pd.DataFrame(registros)
    print("\nColunas detectadas:", df.columns.tolist())
    df = df[[
        "municipio", "codigo_ibge",
        "pib_precos_correntes", "impostos_liquidos", "pib_per_capita",
        "vab_total", "vab_agro", "vab_industria", "vab_servicos", "vab_adm_publica"
    ]]
    out = "data/generated/pib_municipal_serra_penitente.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")
    print(f"\n✅ CSV gerado em: {out}")
    display(f'✅ CSV gerado em: {out}')

## Execução do processamento
Chama a função principal para processar as imagens.

In [19]:
gerar_dados_pib()

Using CPU. Note: This module is much faster with a GPU.



→ Processando pib_alto_parnaiba.png…


p:\ProjectsPython\trabalho_Felipe_CO2\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


→ Texto bruto OCR:
 Alto Parnaiba código: 2100501
PIB a preços correntes
578.991,387 RS {x1coo) [2021]
Impostos; liquidos de subsidios; sobre produtos preços correntes
19926,06 RS (x1ooo) 12021
PIB per capita
51543 79 RS 12021]
Valor adicionado bruto preços correntes
559.065.326 RS {x1coo} 2021l
Agropecuária
362.237,776 Rf (xiooc) [2021
Indústria
34198.51 RS (x1coo) 2021
Serviços Exclusive Administração; defesa educação saúde públicas seguridade social
105145.7 RS (ioco)  12021
Administração; defesa; educação saúde públicas e seguridade social
57.283.34 RS (x1oco) 12021l 

  • Encontrou pib_precos_correntes: 578991.387
  • Encontrou impostos_liquidos: 19926.06
  • Encontrou pib_per_capita: 5154379.0
  • Encontrou vab_total: 559065326.0
  • Encontrou vab_agro: 362237.776
  • Encontrou vab_industria: 3419851.0
  • Encontrou vab_servicos: 1051457.0
  • Encontrou vab_adm_publica: 5728334.0
→ Registro extraído: {'municipio': 'Alto Parnaiba', 'codigo_ibge': '2100501', 'pib_precos_correntes':

p:\ProjectsPython\trabalho_Felipe_CO2\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


→ Texto bruto OCR:
 Balsas código 2101400
PIB a preços correntes
6.307.609.556 RS {x1ooo}   120z1l
Impostos; liquidos de subsidios; sobre produtos; preços correntes
847087,908 RS (x1ooc}  [2021]
PIB per capita
65.059.77 RS 12021]
Valor adicionado bruto preços correntes
5.260.521,648 RS {x1coo} [2021l
Agropecuária
893662,.254 RS (x1ooo) 12021]
Indústria
251291,233 RS {(x1coo} [2021l
Serviços Exclusive Administração; defesa; educação saúde públicas seguridade social
2906.431136 RS {x1ooo} i2021l
Administração; defesa; educação saude públicas seguridade social
409.136 82 RS (xloco)  12o21 

  • Encontrou pib_precos_correntes: 6307609556.0
  • Encontrou impostos_liquidos: 847087.908
  • Encontrou pib_per_capita: 6505977.0
  • Encontrou vab_total: 5260521.648
  • Encontrou vab_agro: 893662.254
  • Encontrou vab_industria: 251291.233
  • Encontrou vab_servicos: 2906431136.0
  • Encontrou vab_adm_publica: 40913682.0
→ Registro extraído: {'municipio': 'Balsas', 'codigo_ibge': '2101400', 'pib_p

p:\ProjectsPython\trabalho_Felipe_CO2\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


→ Texto bruto OCR:
 Tasso Fragoso código: 2112001
PIB a preços correntes
2351251,559 RS (x1coo) [2021l
Impostos; liquidos de subsidios; sobre produtos; preços correntes
54.579 928 RS {xicoo} [2021]
PIB per capita
272 095.76 RS [2021]
Valor adicionado bruto preços correntes
2296.871,631 Rf (x1ooc) 12021]
Agropecuária
1863065.762 RS {x1ooo} i2021l
Indústria
67993,254 RS {x1ooo} 12021/
Serviços Exclusive Administração; defesa; educação saúde públicas seguridade social
314491132 RS {x1ooo} 12021l
Administração; defesa; educação saúde públicas seguridade social
51321,283 RS (xicoo} [2021] 

  • Encontrou pib_precos_correntes: 2351251.559
  • Encontrou impostos_liquidos: 54579928.0
  • Encontrou pib_per_capita: 27209576.0
  • Encontrou vab_total: 2296871.631
  • Encontrou vab_agro: 1863065762.0
  • Encontrou vab_industria: 67993.254
  • Encontrou vab_servicos: 314491132.0
  • Encontrou vab_adm_publica: 51321.283
→ Registro extraído: {'municipio': 'Tasso Fragoso', 'codigo_ibge': '2112001', 'p

'\n✅ CSV gerado em: data/generated/pib_municipal_serra_penitente.csv'